<a href="https://colab.research.google.com/github/Cousar/Ex01_Q11/blob/main/ExtractTE_FromtheSetofTXTfiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import re
from google.colab import drive

# --- Define the custom sorting key function for Numerical Order ---
def ordered_shift_sort_key(s):
    """
    Key function to sort filenames based on the shift value (e.g., -25 to +25).
    It extracts the first number (including its sign) from the filename.
    """
    # Regex to find the 'shift' number, which can be negative or positive.
    # It looks for an optional sign (+ or -), followed by one or more digits.
    match = re.search(r'([+-]?\d+)', s)

    if match:
        # The first extracted group is the number string (e.g., '-25', '+1', '10')
        shift_value = int(match.group(1).replace('+', '')) # Convert to integer
        # The primary key is the numerical shift value
        return (shift_value, s.lower())
    else:
        # Fallback for filenames that don't match the pattern (e.g., generic names)
        return (float('inf'), s.lower())

# -------------------------------------------------


# -------------------------------------------------
# ⚠️ STEP 1: MOUNT GOOGLE DRIVE (MUST BE RUN FIRST IN A SEPARATE CELL)
# from google.colab import drive
# drive.mount('/content/drive')
# -------------------------------------------------

# -------------------------------------------------
# USER INPUT: MANUALLY SPECIFY THE PATH
# -------------------------------------------------
# IMPORTANT: Replace THIS PATH with the actual path to your folder
folder_path = "/content/drive/Othercomputers/My Laptop/Germany/C.V/Essen/Ph.D/FA Cup Project/RayStation Results/140MeV"

if not os.path.isdir(folder_path):
    raise FileNotFoundError(
        "Please update the 'folder_path' variable with the correct location "
        "and ensure the folder is accessible after mounting Google Drive."
    )

print(f"\nAccessing folder directly at:\n{folder_path}")

# -------------------------------------------------
# EXTRACTION SETTINGS
# -------------------------------------------------
target_key = "Sum_Track_Ends_3,3cmHalf_Aperture:"
output_excel = os.path.join(folder_path, "Sum_Track_Ends_Extraction.xlsx")

# -------------------------------------------------
# DATA EXTRACTION
# -------------------------------------------------
rows = []

# --- MODIFIED: Applying the custom key for specific numerical order ---
# 1. Get all filenames
all_files = os.listdir(folder_path)

# 2. Filter for only .txt files and apply the new ordering key
txt_files = sorted(
    [f for f in all_files if f.lower().endswith(".txt")],
    key=ordered_shift_sort_key # <--- THIS is the key change
)

for filename in txt_files:
    full_path = os.path.join(folder_path, filename)
    file_name_only = os.path.splitext(filename)[0]

    extracted_value = None

    try:
        with open(full_path, "r", encoding="utf-8") as f:
            for line in f:
                if target_key in line:
                    # Extract the value after the colon and convert to float
                    # Handles potential comma as decimal separator, replaces it with dot
                    value_part = line.split(":")[1].strip()
                    extracted_value = float(value_part.replace(',', '.'))
                    break
    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        extracted_value = "EXTRACTION_ERROR"

    rows.append({
        "File name": file_name_only,
        "Sum_Track_Ends_3,3cmHalf_Aperture": extracted_value
    })

# -------------------------------------------------
# WRITE EXCEL INTO SAME DRIVE FOLDER
# -------------------------------------------------
df = pd.DataFrame(rows)
df.to_excel(output_excel, index=False)

print("\nDone.")
print(f"Excel file created and saved to your Google Drive at:\n{output_excel}")


Accessing folder directly at:
/content/drive/Othercomputers/My Laptop/Germany/C.V/Essen/Ph.D/FA Cup Project/RayStation Results/140MeV

Done.
Excel file created and saved to your Google Drive at:
/content/drive/Othercomputers/My Laptop/Germany/C.V/Essen/Ph.D/FA Cup Project/RayStation Results/140MeV/Sum_Track_Ends_Extraction.xlsx


# New Section